In [1]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split

import pandas as pd
import os

In [2]:
ROOT_DIR = "../data"
RANDOM_STATE = 200

train_data = pd.read_csv(os.path.join(ROOT_DIR, "preprocessing_train.csv"))

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

normal_ratio = 1.0

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)

## under sampling
df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)

train_data = df_concat

# df_train, df_val = train_test_split(
#     df_concat,
#     test_size=0.3,
#     stratify=df_concat["target"],
#     random_state=RANDOM_STATE,
# )

In [3]:
setup_clf = setup(data=train_data, target="target", train_size=0.7, session_id=333, fold_shuffle=True)

,Description,Value
0,Session id,333
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(4700, 146)"
5,Transformed data shape,"(4700, 170)"
6,Transformed train set shape,"(3290, 170)"
7,Transformed test set shape,"(1410, 170)"
8,Numeric features,133
9,Categorical features,12


In [4]:
model = compare_models(sort='F1', fold=10, n_select=5)

In [5]:
tuned_model = [tune_model(i) for i in model]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5775,0.6016,0.5775,0.5787,0.5762,0.1553,0.1563
1,0.6201,0.6643,0.6201,0.6202,0.6200,0.2402,0.2403
2,0.6049,0.6243,0.6049,0.6050,0.6048,0.2098,0.2099
3,0.6079,0.6560,0.6079,0.6079,0.6079,0.2158,0.2158
4,0.6353,0.6730,0.6353,0.6358,0.6350,0.2706,0.2711
5,0.5623,0.6212,0.5623,0.5629,0.5609,0.1243,0.1251
6,0.5289,0.5623,0.5289,0.5297,0.5245,0.0572,0.0582
7,0.6748,0.7347,0.6748,0.6749,0.6747,0.3495,0.3496
8,0.5805,0.6180,0.5805,0.5807,0.5803,0.1610,0.1612


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5836,0.5931,0.5836,0.5840,0.5829,0.1670,0.1675
1,0.6474,0.6621,0.6474,0.6523,0.6444,0.2944,0.2995
2,0.5714,0.6070,0.5714,0.5715,0.5714,0.1429,0.1429
3,0.6109,0.6632,0.6109,0.6111,0.6107,0.2218,0.2220
4,0.6383,0.6460,0.6383,0.6409,0.6364,0.2763,0.2790
5,0.6109,0.6180,0.6109,0.6119,0.6103,0.2221,0.2229
6,0.5046,0.5425,0.5046,0.5046,0.5046,0.0091,0.0091
7,0.6717,0.7288,0.6717,0.6750,0.6703,0.3437,0.3468
8,0.5745,0.5949,0.5745,0.5766,0.5719,0.1493,0.1512


Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5684,0.6147,0.5684,0.5687,0.5678,0.1366,0.1369
1,0.6505,0.6827,0.6505,0.6544,0.6480,0.3006,0.3047
2,0.5866,0.6186,0.5866,0.5867,0.5866,0.1733,0.1734
3,0.6231,0.6724,0.6231,0.6242,0.6222,0.2460,0.2471
4,0.6201,0.6582,0.6201,0.6210,0.6192,0.2399,0.2409
5,0.6049,0.6249,0.6049,0.6057,0.6042,0.2099,0.2107
6,0.5471,0.5810,0.5471,0.5482,0.5450,0.0946,0.0955
7,0.7021,0.7357,0.7021,0.7033,0.7018,0.4044,0.4055
8,0.5836,0.6271,0.5836,0.5837,0.5835,0.1672,0.1673


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5775,0.5996,0.5775,0.5781,0.5769,0.1552,0.1557
1,0.6261,0.6483,0.6261,0.6263,0.6259,0.2522,0.2524
2,0.5805,0.5926,0.5805,0.5807,0.5804,0.1612,0.1613
3,0.6292,0.6610,0.6292,0.6292,0.6292,0.2584,0.2584
4,0.6049,0.6343,0.6049,0.6049,0.6049,0.2097,0.2098
5,0.5866,0.5948,0.5866,0.5867,0.5866,0.1733,0.1733
6,0.5441,0.5567,0.5441,0.5441,0.5438,0.0880,0.0881
7,0.6687,0.7309,0.6687,0.6702,0.6680,0.3376,0.3390
8,0.5897,0.6165,0.5897,0.5897,0.5897,0.1793,0.1793


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [6]:
tuned_model

[RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                        class_weight='balanced_subsample', criterion='entropy',
                        max_depth=7, max_features='log2', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0,
                        min_samples_leaf=3, min_samples_split=7,
                        min_weight_fraction_leaf=0.0, monotonic_cst=None,
                        n_estimators=140, n_jobs=-1, oob_score=False,
                        random_state=333, verbose=0, warm_start=False),
 RidgeClassifier(alpha=9.86, class_weight=None, copy_X=True, fit_intercept=False,
                 max_iter=None, positive=False, random_state=333, solver='auto',
                 tol=0.0001),
 LGBMClassifier(bagging_fraction=1.0, bagging_freq=2, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.1, max_depth=-1,
               

In [8]:
#a = [tuned_model[0], tuned_model[], tuned_model[3], tuned_model[4]]

tuned_model.pop(1)

RidgeClassifier(alpha=9.86, class_weight=None, copy_X=True, fit_intercept=False,
                max_iter=None, positive=False, random_state=333, solver='auto',
                tol=0.0001)

In [9]:
blended_soft = blend_models(estimator_list=tuned_model,
                       fold=10,
                       method="hard",
                       optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5866,0.0000,0.5866,0.5872,0.5861,0.1734,0.1739
1,0.6292,0.0000,0.6292,0.6292,0.6291,0.2583,0.2584
2,0.5866,0.0000,0.5866,0.5878,0.5855,0.1735,0.1746
3,0.6079,0.0000,0.6079,0.6080,0.6078,0.2159,0.2160
4,0.6353,0.0000,0.6353,0.6355,0.6352,0.2706,0.2708
5,0.5684,0.0000,0.5684,0.5688,0.5675,0.1365,0.1370
6,0.5350,0.0000,0.5350,0.5357,0.5317,0.0694,0.0704
7,0.6626,0.0000,0.6626,0.6626,0.6626,0.3252,0.3252
8,0.5897,0.0000,0.5897,0.5899,0.5893,0.1792,0.1795


In [10]:
final_model = finalize_model(blended_soft)
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [11]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "preprocessing_test.csv"))

train_columns = train_data.columns
test_data = test_data[train_columns]

df_test_x = test_data.drop(columns=['target'])

In [12]:
test_pred = predict_model(final_model, data=df_test_x)
test_pred

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,prediction_label
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,90,10,...,91.800003,270.000000,50,85,19.799999,13.000000,195,1,0,AbNormal
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,14,256,1,Normal
2,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,-90,10,...,91.800003,270.000000,50,85,19.700001,1.000000,98,1,0,AbNormal
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,90,10,...,50.000000,91.800003,270,50,85.000000,20.000000,14,0,1,Normal
4,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,-90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,1,215,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,90,10,...,50.000000,91.800003,270,50,85.000000,19.500000,14,131,1,AbNormal
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,12,279,1,Normal
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,-90,16,...,50.000000,91.800003,270,50,85.000000,20.500000,4,66,1,AbNormal
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,-90,10,...,91.800003,270.000000,50,85,18.900000,1.000000,117,1,0,AbNormal


In [13]:
df_sub = pd.read_csv("../data/submission.csv")
df_sub["target"] = test_pred["prediction_label"]
df_sub.to_csv("../data/automl/submission-5.csv", index=False)

In [14]:
final_model

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['CURE END POSITION X Collect '
                                             'Result_Dam',
                                             'CURE END POSITION Z Collect '
                                             'Result_Dam',
                                             'CURE END POSITION Θ Collect '
                                             'Result_Dam',
                                             'CURE SPEED Collect Result_Da...
                                                                          max_features='log2',
                                                                          max_leaf_nodes=None,
                                                                          min_impurity_decrease=0.001,
                                                                          min_samples_leaf=4,
                                                                          min_samples_split=10,
                                                                          min_weight_fraction_leaf=0.0,
                                                                          n_estimators=10,
                                                                          n_iter_no_change=None,
                                                                          random_state=333,
                                                                          subsample=0.4,
                                                                          tol=0.0001,
                                                                          validation_fraction=0.1,
                                                                          verbose=0,
                                                                          warm_start=False))],
                                  flatten_transform=True, n_jobs=-1,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)